# Deployment Model

In [20]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.metrics import cohen_kappa_score
import time

In [21]:
SCORE_RANGES = {
        1: {'sentence_fluency': (1, 6), 'word_choice': (1, 6), 'conventions': (1, 6),'organization': (1, 6),
            'content': (1, 6), 'holistic': (2, 12)},
        2: {'sentence_fluency': (1, 6), 'word_choice': (1, 6), 'conventions': (1, 6),'organization': (1, 6),
            'content': (1, 6), 'holistic': (1, 6)},
        3: {'narrativity': (0, 3), 'language': (0, 3), 'prompt_adherence': (0, 3), 'content': (0, 3), 'holistic': (0, 3)},
        4: {'narrativity': (0, 3), 'language': (0, 3), 'prompt_adherence': (0, 3), 'content': (0, 3), 'holistic': (0, 3)},
        5: {'narrativity': (0, 4), 'language': (0, 4), 'prompt_adherence': (0, 4), 'content': (0, 4), 'holistic': (0, 4)},
        6: {'narrativity': (0, 4), 'language': (0, 4), 'prompt_adherence': (0, 4), 'content': (0, 4), 'holistic': (0, 4)},
        7: {'conventions': (0, 6), 'organization': (0, 6), 'content': (0, 6),'holistic': (0, 30)},
        8: {'sentence_fluency': (2, 12), 'word_choice': (2, 12), 'conventions': (2, 12),'organization': (2, 12),
            'content': (2, 12), 'holistic': (0, 60)}}

def read_data(path):
    """Read the CSV file and return a dictionary of data"""
    data = pd.read_csv(path)
    data_dict = {
        'essay_ids': data['essay_id'].values,
        'prompt_ids': data['prompt_id'].values,
        'essay_text': data['essay_text'].values,
        'features': data.iloc[:, 12:].values,
        'holistic': data['holistic'].values
    }
    return data_dict

def quadratic_weighted_kappa(y_true, y_pred):
    """Calculate QWK score between true and predicted labels"""
    return cohen_kappa_score(y_true, np.round(y_pred), weights='quadratic')


In [22]:
def normalize_scores(scores, prompt_id):
    score_range = SCORE_RANGES[prompt_id]['holistic']
    return (scores - score_range[0]) / (score_range[1] - score_range[0])

def denormalize_scores(norm_scores, prompt_id):
    score_range = SCORE_RANGES[prompt_id]['holistic']
    return norm_scores * (score_range[1] - score_range[0]) + score_range[0]


In [23]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_unit, num_layers):
        super(NeuralNetwork, self).__init__()
        layers = []
        layers.append(nn.Linear(input_size, hidden_unit))
        layers.append(nn.ReLU())
        for i in range(num_layers - 1):
            layers.append(nn.Linear(hidden_unit, hidden_unit))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_unit, 1))
        self.model = nn.Sequential(*layers)

        for i in self.modules():
            if isinstance(i, nn.Linear):
                nn.init.kaiming_normal_(i.weight)
                nn.init.zeros_(i.bias)

    def forward(self, x):
        return self.model(x)


In [24]:
class DeploymentModel(nn.Module):
    def __init__(self, base_model):
        super(DeploymentModel, self).__init__()
        self.model = base_model

    def forward(self, features, prompt_ids):
        # Get normalized predictions
        normalized_preds = self.model(features)

        # Denormalize for each prompt_id
        batch_size = features.shape[0]
        denormalized_preds = torch.zeros_like(normalized_preds)

        for i in range(batch_size):
            denormalized_preds[i] = denormalize_scores(
                normalized_preds[i],
                prompt_ids[i].item()
            )

        return denormalized_preds

In [25]:
def main():
    print(f"\n{'-'*50}")
    print(f"deployment model training started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"{'-'*50}\n")

    print("Reading dataset\n")
    data = read_data('dataset.csv')

    features = torch.FloatTensor(data['features'])
    scores = torch.FloatTensor(data['holistic'])
    prompt_ids = data['prompt_ids']

    print(f"Dataset loaded: {len(features)} essays, {features.shape[1]} features")

    normalized_scores = torch.zeros_like(scores)
    for prompt_id in range(1, 9):
        prompt_filter = prompt_ids == prompt_id
        normalized_scores[prompt_filter] = normalize_scores(scores[prompt_filter], prompt_id)
    normalized_scores = normalized_scores.reshape(-1, 1)

    final_params = {
        'hidden_unit': 32,
        'num_layers': 2,
        'learning_rate': 0.001,
        'batch_size': 32
    }

    print("\ninitializing model with best parameters:")
    for param, value in final_params.items():
        print(f"{param}: {value}")

    model = NeuralNetwork(
        input_size=86,
        hidden_unit=final_params['hidden_unit'],
        num_layers=final_params['num_layers']
    )

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=final_params['learning_rate'],
        betas=(0.9, 0.999),
        weight_decay=0.1
    )
    criterion = nn.MSELoss()

    best_loss = float('inf')
    epochs_no_improve = 0
    batch_size = final_params['batch_size']
    start_time = time.time()

    for epoch in range(15):
        model.train()
        epoch_loss = 0
        num_batches = 0

        for i in range(0, len(features), batch_size):
            batch_X = features[i:i + batch_size]
            batch_y = normalized_scores[i:i + batch_size]

            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            num_batches += 1

        avg_epoch_loss = epoch_loss / num_batches
        print(f"Epoch {epoch + 1}/15 - Loss: {avg_epoch_loss:.6f}")

        if avg_epoch_loss < best_loss:
            best_loss = avg_epoch_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= 3:
            print(f"\nEarly stopping at epoch {epoch + 1}")
            break

    training_time = time.time() - start_time

    print(f"\n{'-'*50}")
    print("training completed successfully!  :)")
    print(f"Training time: {training_time/60:.2f} minutes")
    print(f"Final loss: {best_loss:.6f}")
    print(f"{'-'*50}\n")

    # Save the base model (outputs normalized scores)
    scripted_model = torch.jit.script(model)
    scripted_model.save("model-A-deployment.pt")
    print("\nModel saved as 'model-A-deployment.pt'")

    #verify predictions with manual denormalization
    print("\nVerifying predictions:")
    with torch.no_grad():
        test_batch = features[:5]
        test_prompts = prompt_ids[:5]
        normalized_preds = model(test_batch)

        #manually denormalize
        denormalized_preds = torch.zeros_like(normalized_preds)
        for i in range(len(test_batch)):
            denormalized_preds[i] = denormalize_scores(normalized_preds[i], test_prompts[i])

        print("Sample predictions (denormalized):", denormalized_preds.numpy().flatten())
        print("Original scores:", scores[:5])

if __name__ == "__main__":
    main()


--------------------------------------------------
deployment model training started at 2024-12-07 13:29:30
--------------------------------------------------

Reading dataset

Dataset loaded: 12976 essays, 86 features

initializing model with best parameters:
hidden_unit: 32
num_layers: 2
learning_rate: 0.001
batch_size: 32
Epoch 1/15 - Loss: 0.032144
Epoch 2/15 - Loss: 0.022328
Epoch 3/15 - Loss: 0.021579
Epoch 4/15 - Loss: 0.021213
Epoch 5/15 - Loss: 0.020940
Epoch 6/15 - Loss: 0.020660
Epoch 7/15 - Loss: 0.020448
Epoch 8/15 - Loss: 0.020271
Epoch 9/15 - Loss: 0.020153
Epoch 10/15 - Loss: 0.020071
Epoch 11/15 - Loss: 0.020016
Epoch 12/15 - Loss: 0.019867
Epoch 13/15 - Loss: 0.019806
Epoch 14/15 - Loss: 0.019685
Epoch 15/15 - Loss: 0.019574

--------------------------------------------------
training completed successfully!  :)
Training time: 0.16 minutes
Final loss: 0.019574
--------------------------------------------------


Model saved as 'model-A-deployment.pt'

Verifying predi